In [1]:
#pip install gmaps

In [2]:

# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [7]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Klaksvik,62.2266,-6.5890,54.52,79,97,13.82,FO,2022-08-29 15:13:47
1,1,Ostrov,57.3439,28.3537,80.33,56,94,5.01,RU,2022-08-29 15:13:47
2,2,Cherskiy,68.7500,161.3000,44.71,89,7,7.92,RU,2022-08-29 15:13:48
3,3,Jimo,36.3897,120.4622,65.32,95,100,3.13,CN,2022-08-29 15:13:48
4,4,Raymond,43.9015,-70.4703,82.80,76,89,5.93,US,2022-08-29 15:13:48


In [8]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [9]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [21]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [16]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [34]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [36]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       1
Date          0
dtype: int64

In [37]:
preferred_cities_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Ostrov,57.3439,28.3537,80.33,56,94,5.01,RU,2022-08-29 15:13:47
4,4,Raymond,43.9015,-70.4703,82.80,76,89,5.93,US,2022-08-29 15:13:48
5,5,Kavaratti,10.5669,72.6420,80.98,81,100,19.69,IN,2022-08-29 15:13:49
9,9,Hami,42.8000,93.4500,77.95,16,1,3.36,CN,2022-08-29 15:13:12
13,13,Omboue,-1.5746,9.2618,78.53,71,96,8.90,GA,2022-08-29 15:13:51
...,...,...,...,...,...,...,...,...,...,...
558,558,Cuorgne,45.3895,7.6499,87.66,55,25,1.01,IT,2022-08-29 15:28:38
559,559,Ishigaki,24.3448,124.1572,84.33,89,40,4.61,JP,2022-08-29 15:28:39
560,560,Broome,42.2506,-75.8330,83.01,74,100,10.00,US,2022-08-29 15:28:39
564,564,Takoradi,4.8845,-1.7554,77.72,77,88,9.64,GH,2022-08-29 15:27:02


In [38]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ostrov,RU,80.33,57.3439,28.3537,
4,Raymond,US,82.80,43.9015,-70.4703,
5,Kavaratti,IN,80.98,10.5669,72.6420,
9,Hami,CN,77.95,42.8000,93.4500,
13,Omboue,GA,78.53,-1.5746,9.2618,
14,Kapaa,US,77.32,22.0752,-159.3190,
19,Faya,SA,79.32,18.3851,42.4509,
23,Maragogi,BR,78.98,-9.0122,-35.2225,
26,Ribeira Grande,PT,77.40,38.5167,-28.7000,
30,Victoria,HK,84.38,22.2855,114.1577,


In [39]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [42]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ostrov,RU,80.33,57.3439,28.3537,Ostrov-Park
4,Raymond,US,82.80,43.9015,-70.4703,Migis Lodge at Sebago Lake
5,Kavaratti,IN,80.98,10.5669,72.6420,Green leaf
9,Hami,CN,77.95,42.8000,93.4500,Laoyutang Resort
13,Omboue,GA,78.53,-1.5746,9.2618,Hotel Olako
14,Kapaa,US,77.32,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Faya,SA,79.32,18.3851,42.4509,املاك خالد الشعشوع
23,Maragogi,BR,78.98,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
26,Ribeira Grande,PT,77.40,38.5167,-28.7000,Quinta da Meia Eira
30,Victoria,HK,84.38,22.2855,114.1577,Mini Central


In [51]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))